In [18]:
import os
import pickle
from ultralytics import YOLO
from PIL import Image

In [19]:
user = os.environ['USER']
model_path = f"/home/{user}/runs/run_nr1/weights/best.pt"
model = YOLO(model_path)

In [20]:
'''
cwd = os.getcwd()
testset_dir = os.path.join(cwd,'..','testset')
label_path = os.path.join(testset_dir,'labels')
exp='8y=40z'
pickle.dump(exp, open(f'{label_path}/34.pkl', 'wb'))
'''

"\ncwd = os.getcwd()\ntestset_dir = os.path.join(cwd,'..','testset')\nlabel_path = os.path.join(testset_dir,'labels')\nexp='8y=40z'\npickle.dump(exp, open(f'{label_path}/34.pkl', 'wb'))\n"

In [21]:
cwd = os.getcwd()
testset_dir = os.path.join(cwd,'..','testset')

sorted_imgs = sorted(os.listdir(os.path.join(testset_dir,'images')), key=lambda x: int(x.split(".")[0]))
sorted_labels = sorted(os.listdir(os.path.join(testset_dir,'labels')), key=lambda x: int(x.split(".")[0]))
print(sorted_imgs)
print(sorted_labels)

imgs=[]
labels=[]
for i in sorted_imgs:
    print(i)
    img = Image.open(os.path.join(testset_dir,'images',i))
    imgs.append(img)

for i in sorted_labels:
    print(i)
    label = pickle.load(open(os.path.join(testset_dir,'labels',i),"rb"))
    #print(label)
    labels.append(label)


['0.png', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png', '31.png', '32.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '40.png', '41.png', '42.png', '43.png', '44.png', '45.png', '46.png', '47.png', '48.png', '49.png', '50.png', '51.png', '52.png', '53.png', '54.png', '55.png']
['0.pkl', '1.pkl', '2.pkl', '3.pkl', '4.pkl', '5.pkl', '6.pkl', '7.pkl', '8.pkl', '9.pkl', '10.pkl', '11.pkl', '12.pkl', '13.pkl', '14.pkl', '15.pkl', '16.pkl', '17.pkl', '18.pkl', '19.pkl', '20.pkl', '21.pkl', '22.pkl', '23.pkl', '24.pkl', '25.pkl', '26.pkl', '27.pkl', '28.pkl', '29.pkl', '30.pkl', '31.pkl', '32.pkl', '34.pkl', '35.pkl', '36.pkl', '37.pkl', '38.pkl', '39.pkl', '40.pkl', '41.pkl', '42.pkl', '43.pkl', '44.pkl', '45.pkl', '46.pkl', '47.pkl'

In [22]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
import green_tsetlin as gt

data_dir = f"/home/{user}/data/math_expression_data/tm_data"
tm_save_path = os.path.join(data_dir, "tm_state_v1.npz")
ds = gt.DenseState.load_from_file(tm_save_path)
    
rs = gt.RuleSet(False)
rs.compile_from_dense_state(ds)

print(rs.rules[0])
print(rs.weights[0])

predictor = gt.Predictor(explanation="none", multi_label=False)
predictor._set_ruleset(rs)
predictor._allocate_backend()

class_names_map = {0: '*', 1: 'tan', 2: '(', 3: '6', 4: 'y', 5: '-', 6: '2', 7: '/', 8: '7', 9: '8', 10: ')', 11: '4', 12: 'sqrt', 13: '=', 14: '0', 15: 'sin', 16: '3', 17: '+', 18: '1', 19: 'cos', 20: 'z', 21: '9', 22: 'pi', 23: '5'}


def classify_image(test_image):
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

    h, w = test_image.shape[:2]
    aspect = w / h

    test_image = cv2.threshold(test_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # PADDING
    if aspect != 1:
        if aspect > 1:
            pad_vert = (w - h) // 2
            pad_horiz = 0
        else:
            pad_vert = 0
            pad_horiz = (h - w) // 2
        
        # Pad the image to make it square
        test_image = cv2.copyMakeBorder(test_image, pad_vert, pad_vert, pad_horiz, pad_horiz, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    
    test_image = cv2.resize(test_image, (45, 45), interpolation=cv2.INTER_AREA)
    skeleton = skeletonize(test_image)
    
    skeleton = skeleton.astype(int)
    
    pred_image = skeleton.flatten()
    
    # Predict the class of the test
    predicted_class = predictor.predict(pred_image)
    
    return class_names_map[predicted_class]

[2061, 2174, 2285, 2292, 2328, 2647, 2678, 2679, 2682, 2703, 2725, 2774, 2848, 2858, 2901, 2909, 2948, 2982, 2985, 2988, 2990, 3012, 3046, 3047, 3052, 3060, 3152, 3167, 3168, 3173, 3245, 3290, 3324, 3360, 3436, 3564, 3671, 3740, 3746, 3758, 3844, 3873, 3903, 3904, 3963, 3998]
[-66, 8, -10, -19, -29, -5, -102, -56, -27, -23, 65, 9, -31, 54, 108, -43, 36, -52, -46, -52, -37, -4, 5, -33, -11, -18, -15, -11, -13, -11, -16, -7, -19, -17, -11, -22, -15, -13, -4, -11, -19]


In [23]:
corr=0
for i, img in enumerate(imgs):
    result = model.predict(source=img, conf=0.5)[0]

    img_array = result.orig_img

    imgs = []
    for res in result:

        box = res.boxes.xyxy[0].cpu().numpy().astype(int)

        img = img_array[box[1]:box[3], box[0]:box[2]]
        imgs.append([img, box])
        
    imgs = sorted(imgs, key=lambda x: x[1][0])
    
    math_expression = ""

    for img, box in imgs:
        im = Image.fromarray(img)

        # predict the class of the image using the TM
        tm_res = classify_image(img)
        math_expression += tm_res

        # uncomment to display the images that are being detected and classified
        '''
        display(im)
        print(img.dtype)
        print(tm_res)
        '''
    print('PREDICTED LABEL:',math_expression)
    true_label = labels[i]
    print('TRUE LABEL:',true_label)
    if math_expression == true_label:
        corr+=1

print('ACCURACY:',corr/len(imgs))


0: 96x640 2 +s, 1 x, 2 4s, 2 6s, 1 y, 1 z, 1 \\left(, 3 \\right)s, 13.9ms
Speed: 0.3ms preprocess, 13.9ms inference, 0.5ms postprocess per image at shape (1, 3, 96, 640)
PREDICTED LABEL: (66+44)*(y+z)
TRUE LABEL: (66+44)*(y+z)

0: 96x640 1 4, 1 8, 1 z, 2 \\left(s, 2 \\right)s, 1 \\cos, 1 \\tan, 13.4ms
Speed: 0.4ms preprocess, 13.4ms inference, 0.6ms postprocess per image at shape (1, 3, 96, 640)
PREDICTED LABEL: tan(+)8cos(1)
TRUE LABEL: tan(4)+8=cos(z)

0: 128x640 1 +, 1 2, 1 4, 1 y, 2 \\left(s, 2 \\right)s, 1 \\tan, 15.4ms
Speed: 0.4ms preprocess, 15.4ms inference, 0.5ms postprocess per image at shape (1, 3, 128, 640)
PREDICTED LABEL: (4)+tan(2y)
TRUE LABEL: sqrt(4)+tan(2y)

0: 96x640 1 4, 1 8, 1 z, 2 \\left(s, 2 \\right)s, 1 \\cos, 1 \\tan, 14.3ms
Speed: 0.4ms preprocess, 14.3ms inference, 0.6ms postprocess per image at shape (1, 3, 96, 640)
PREDICTED LABEL: tan(4)8cos(z)
TRUE LABEL: tan(4)+8=cos(z)

0: 352x640 1 z, 31.9ms
Speed: 1.0ms preprocess, 31.9ms inference, 0.6ms postproces

In [24]:
img_array = result.orig_img

imgs = []
for res in result:

    box = res.boxes.xyxy[0].cpu().numpy().astype(int)

    img = img_array[box[1]:box[3], box[0]:box[2]]
    imgs.append([img, box])
    

# sort the images by the x coordinate
imgs = sorted(imgs, key=lambda x: x[1][0])

# display the images
for img, box in imgs:
    im = Image.fromarray(img)
    
    im.save(f"/home/eirikmv/cv_project2/test_imgs/{box[0]}_{box[1]}.png")

FileNotFoundError: [Errno 2] No such file or directory: '/home/eirikmv/cv_project2/test_imgs/12_51.png'